### Imports

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
import numpy as np
from neuropy.utils.neurons_util import calculate_neurons_ccg,ccg_temporal_bias
import pingouin as pg
from tqdm import tqdm
import subjects

### Temporal bias maze vs remaze

In [ ]:
sessions = subjects.remaze_sess()

In [ ]:
bias_df = []
for sub, sess in enumerate(tqdm(sessions)):
    pre = sess.paradigm["pre"].flatten()
    maze = sess.paradigm["maze"].flatten()
    remaze = sess.paradigm["re-maze"].flatten()

    neurons = sess.neurons_stable.get_neuron_type("pyr")

    get_bias = lambda epoch: ccg_temporal_bias(
        neurons.time_slice(epoch[0], epoch[1])
    )

    pre_bias = get_bias(pre)
    maze_bias = get_bias(maze)
    remaze_bias = get_bias(remaze)

    bias_df.append(
        pd.DataFrame(
            dict(pre=pre_bias, maze=maze_bias, remaze=remaze_bias, grp=sess.tag)
        )
    )

bias_df = pd.concat(bias_df, ignore_index=True)
subjects.GroupData().save(bias_df, "remaze_temporal_bias")

In [ ]:
import seaborn as sns
from neuropy.plotting import Fig

fig = Fig(grid=(5,5))

for g,grp in enumerate(['NSD','SD']):
    df = bias_df[(bias_df['grp']==grp)]
    x = df['maze'].values
    for e, epoch in enumerate(['pre','remaze']):
        ax = fig.subplot(fig.gs[0,e])
        y = df[epoch].values

        mask = ~np.isnan(x) & ~np.isnan(y)
        linfit = stats.linregress(x[mask],y[mask])
        print(linfit.rvalue)
        ax.scatter(x,y,s=5,alpha=0.6)


### Temporal bias center of mass maze vs remaze
- using CCG temporal bias but uses center of mass instead of difference in number of spikes (Gerrard2008)

In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import subjects

sessions = subjects.nsd.remaze + subjects.sd.remaze

In [ ]:
from neuropy.analyses import ExplainedVariance
import pingouin as pg
from neuropy.utils.neurons_util import ccg_temporal_bias


bias_df = []
for sub, sess in enumerate(tqdm(sessions)):
    pre = sess.paradigm["pre"].flatten()
    maze = sess.paradigm["maze"].flatten()
    remaze = sess.paradigm["re-maze"].flatten()

    neurons = sess.neurons.get_neuron_type("pyr")
    # wave_similarity = neurons.get_waveform_similarity()
    # pairs_bool = wave_similarity < 0.8  # only pairs which are least similar
    tb = lambda e: ccg_temporal_bias(neurons.time_slice(*e),com=True,window_size=0.5)

    pre_bias = tb(pre)
    maze_bias =tb(maze)
    remaze_bias =tb(remaze)

    bias_df.append(
        pd.DataFrame(
            dict(pre=pre_bias, maze=maze_bias, remaze=remaze_bias, grp=sess.tag)
        )
    )

bias_df = pd.concat(bias_df, ignore_index=True)
# subjects.GroupData().save(bias_df, "remaze_temporal_bias_com")


In [ ]:
%matplotlib widget
import seaborn as sns

sns.lmplot(
    data=bias_df,
    x="maze",
    y="remaze",
    hue="grp",
    palette=["#9C27B0", "#F06292"],
    # x_partial="pre",
    # y_partial="pre",
    scatter_kws={'s':8},
    # col='grp',
)


### COM evolution as a function of time in POST (SD vs NSD)

In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import subjects

sessions = subjects.nsd.pf_sess + subjects.sd.pf_sess

In [ ]:
from neuropy.analyses import ExplainedVariance
import pingouin as pg
from neuropy.utils.neurons_util import ccg_temporal_bias


zts = np.array([0, 2, 4, 6, 8])
bias_df = []
for sub, sess in enumerate(tqdm(sessions)):
    post = sess.paradigm["post"].flatten()

    neurons = sess.neurons.get_neuron_type("pyr")
    # wave_similarity = neurons.get_waveform_similarity()
    # pairs_bool = wave_similarity < 0.8  # only pairs which are least similar
    tb = lambda e: ccg_temporal_bias(neurons.time_slice(*e), com=True, window_size=1.0)

    t_starts = zts * 3600 + post[0]

    bias = []
    for i, t in enumerate(t_starts):
        bias.append(tb([t, t + 3600]))
    bias = np.asarray(bias).T
    
    df = pd.DataFrame(bias, columns=zts)
    df["grp"] = sess.tag
    df["name"] = sess.animal.name

    bias_df.append(df)

bias_df = pd.concat(bias_df, ignore_index=True)
bias_df = pd.melt(
    bias_df,
    id_vars=["grp", "name"],
    value_vars=zts,
    var_name="zt",
    value_name="bias",
    ignore_index=True,
)
# subjects.GroupData().save(bias_df, "remaze_temporal_bias_com")


In [ ]:
%matplotlib widget
import seaborn as sns

sns.boxplot(
    data=bias_df,
    x="zt",
    y="bias",
    hue="grp",
    palette=["#9C27B0", "#F06292"],
    # split=True,
    showfliers=False,
)


### COM correlation between time windows as a function of time in POST (SD vs NSD)
- To visualize relative timing of firing between cell-paris using CCG and how that changes between SD and NSD and observe how the relative firing changes across time.
- The results showed similar stuff to sliding template, time-window correlation i.e, sleep deprivation period is very self similar and very different from maze

In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import subjects

sessions = subjects.nsd.pf_sess + subjects.sd.pf_sess

In [ ]:
from neuropy.analyses import ExplainedVariance
import pingouin as pg
from neuropy.utils.neurons_util import ccg_temporal_bias


zts = np.arange(9)
corr_nsd,corr_sd = [],[]
for sub, sess in enumerate(tqdm(sessions)):
    pre=sess.paradigm['pre'].flatten()
    maze = sess.paradigm["maze"].flatten()
    post = sess.paradigm["post"].flatten()
    neurons = sess.neurons.get_neuron_type("pyr")

    tb = lambda e: ccg_temporal_bias(neurons.time_slice(*e), com=True, window_size=2.0)

    zt_starts = zts * 3600 + post[0]
    zt_epochs = np.vstack((zt_starts,zt_starts+3600)).T
    epochs = np.insert(zt_epochs,0,maze,axis=0)

    bias = []
    for e in epochs:
        bias.append(tb(e))
    bias = np.asarray(bias)
    corr = np.ma.corrcoef(np.ma.masked_invalid(bias))
    corr = np.ma.getdata(corr)
    np.fill_diagonal(corr,0)

    if sess.tag =='nsd':
        corr_nsd.append(corr)
    if sess.tag =='sd':
        corr_sd.append(corr)



In [ ]:
%matplotlib widget
import seaborn as sns
from neuropy.plotting import Fig

fig = Fig(grid=(1, 2))

epoch_names = ["maze"]
epoch_names.extend([f"Zt{_}-{_+1}" for _ in zts])

for i, corrs in enumerate([corr_nsd, corr_sd]):
    ax = fig.subplot(fig.gs[i])
    ax.imshow(
        np.nanmean(np.dstack(corrs), axis=-1),
        cmap="bwr",
        vmin=-0.1,
        vmax=0.1,
        origin="lower",
    )
    ax.set_xlabel("Epochs")
    ax.set_ylabel("Epochs")
    ax.set_xticks(np.arange(8), epoch_names)
    ax.set_yticks(np.arange(8), epoch_names)


### COM (with remaze) correlation between time windows as a function of time in POST (SD vs NSD)
- To visualize relative timing of firing between cell-paris using CCG and how that changes between SD and NSD and observe how the relative firing changes across time.
- The results showed similar stuff to sliding template, time-window correlation i.e, sleep deprivation period is very self similar and very different from maze

In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import subjects

sessions = subjects.nsd.remaze + subjects.sd.remaze

In [ ]:
from neuropy.analyses import ExplainedVariance
import pingouin as pg
from neuropy.utils.neurons_util import ccg_temporal_bias


zts = np.arange(8)
corr_nsd, corr_sd = [], []
for sub, sess in enumerate(tqdm(sessions)):
    pre = sess.paradigm["pre"].flatten()
    maze = sess.paradigm["maze"].flatten()
    post = sess.paradigm["post"].flatten()
    remaze = sess.paradigm["re-maze"].flatten()
    neurons = sess.neurons.get_neuron_type("pyr")

    tb = lambda e: ccg_temporal_bias(neurons.time_slice(*e), com=True, window_size=0.5)

    zt_starts = zts * 3600 + post[0]
    zt_epochs = np.vstack((zt_starts, zt_starts + 3600)).T

    epochs = np.vstack([pre, maze, zt_epochs, remaze])

    bias = []
    for e in epochs:
        bias.append(tb(e))
    bias = np.asarray(bias)
    corr = np.ma.corrcoef(np.ma.masked_invalid(bias))
    corr = np.ma.getdata(corr)
    np.fill_diagonal(corr, np.nan)

    if sess.tag == "NSD":
        corr_nsd.append(corr)
    if sess.tag == "SD":
        corr_sd.append(corr)


epoch_names = ["pre", "maze"]
epoch_names.extend([f"Zt{_}-{_+1}" for _ in zts])
epoch_names.extend(["remaze"])

corr = dict(
    NSD=np.dstack(corr_nsd).mean(axis=-1),
    SD=np.dstack(corr_sd).mean(axis=-1),
    epoch_names=epoch_names,
)
subjects.GroupData().save(corr, "remaze_temporal_bias_com_correlation_across_session")


In [ ]:
%matplotlib widget
import seaborn as sns
from neuropy.plotting import Fig

fig = Fig(grid=(1, 2))

for i, corrs in enumerate([corr_nsd, corr_sd]):
    ax = fig.subplot(fig.gs[i])
    ax.imshow(
        np.nanmean(np.dstack(corrs), axis=-1),
        cmap="jet",
        vmin=-0.1,
        vmax=0.1,
        origin="lower",
    )
    ax.set_xlabel("Epochs")
    ax.set_ylabel("Epochs")
    ax.set_xticks(np.arange(11), epoch_names)
    ax.set_yticks(np.arange(11), epoch_names)
